### **Setup**

In [1]:
%%bash
pip install timm -q

### **Library Imports**

In [2]:
import os
import re
import cv2
import timm
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from time import time
from typing import Union
from torch import nn, optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL
from torchvision import transforms

# from sklearn.model_selection import KFold

### **Utilities and Helpers**

In [3]:
SEED: int = 42
SIZE: int = 384


def breaker(num: int=50, char: str="*") -> None: print("\n" + num*char + "\n")

    
def get_image(path: str, size: int=224) -> np.ndarray:
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB)
    return cv2.resize(src=image, dsize=(size, size), interpolation=cv2.INTER_AREA)

    
# def show_loss_graphs(L: list) -> None:
#     TL, VL = [], []
#     for i in range(len(L)):
#         TL.append(L[i]["train"])
#         VL.append(L[i]["valid"])
#     x_Axis = np.arange(1, len(TL) + 1)
#     plt.figure()
#     plt.plot(x_Axis, TL, "r", label="Train")
#     plt.plot(x_Axis, VL, "b", label="Valid")
#     plt.legend()
#     plt.grid()
#     plt.title("Loss Graph")
#     plt.show()

    
# def show_accuracy_graphs(A: list) -> None:
#     TA, VA = [], []
#     for i in range(len(A)):
#         TA.append(A[i]["train"])
#         VA.append(A[i]["valid"])
#     x_Axis = np.arange(1, len(TA) + 1)
#     plt.figure()
#     plt.plot(x_Axis, TA, "r", label="Train")
#     plt.plot(x_Axis, VA, "b", label="Valid")
#     plt.legend()
#     plt.grid()
#     plt.title("Accuracy Graph")
#     plt.show()
    

# def show_lr_graph(LR: list) -> None:
#     x_Axis = [i+1 for i in range(len(LR))]
#     plt.figure(figsize=(8, 6))
#     plt.plot(x_Axis, LR, "rx")
#     plt.grid()
#     plt.show()



### **Configuration**

In [4]:
class CFG(object):
    def __init__(self, 
                 seed: int = 42,
                 size: int = 224,
                 n_splits: int = 5,
                 batch_size: int = 16,
                 epochs: int = 25,
                 early_stopping: int = 5,
                 lr: float = 1e-4,
                 wd: float = 0.0,
                 max_lr: float = 1e-3,
                 pct_start: float = 0.2,
                 steps_per_epoch: int = 100,
                 div_factor: int = 1e3, 
                 final_div_factor: float = 1e3,
                 ):
        self.seed = seed
        self.size = size
        self.n_splits = n_splits
        self.batch_size = batch_size
        self.epochs = epochs
        self.early_stopping = early_stopping
        self.lr = lr
        self.wd = wd
        self.max_lr = max_lr
        self.pct_start = pct_start
        self.steps_per_epoch = steps_per_epoch
        self.div_factor = div_factor
        self.final_div_factor = final_div_factor
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        if self.size == 224:
            self.train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.36468, 0.33754, 0.31026], [0.20133, 0.19425, 0.19649]),
                transforms.RandomAffine(degrees=(-45, 45), translate=(0.15, 0.15), scale=(0.5, 1.5)),
                transforms.RandomHorizontalFlip(p=0.25),
                transforms.RandomVerticalFlip(p=0.25),
            ])
            self.valid_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.36468, 0.33754, 0.31026], [0.20133, 0.19425, 0.19649]),
            ])
        
        if self.size == 384:
            self.train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.36468, 0.33754, 0.31026], [0.20285, 0.19578, 0.19797]),
                transforms.RandomAffine(degrees=(-45, 45), translate=(0.15, 0.15), scale=(0.5, 1.5)),
                transforms.RandomHorizontalFlip(p=0.25),
                transforms.RandomVerticalFlip(p=0.25),
            ])
            self.valid_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.36468, 0.33754, 0.31026], [0.20285, 0.19578, 0.19797]),
            ])
        
        if self.size == 512:
            self.train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.36468, 0.33754, 0.31026], [0.20326, 0.19620, 0.19838]),
                transforms.RandomAffine(degrees=(-45, 45), translate=(0.15, 0.15), scale=(0.5, 1.5)),
                transforms.RandomHorizontalFlip(p=0.25),
                transforms.RandomVerticalFlip(p=0.25),
            ])
            self.valid_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.36468, 0.33754, 0.31026], [0.20326, 0.19620, 0.19838]),
            ])
                                
        self.save_path = "saves"
        if not os.path.exists(self.save_path): os.makedirs(self.save_path)
    
cfg = CFG(
    seed=SEED, 
    size=SIZE
)

### **Dataset Template**

In [5]:
class DS(Dataset):
    def __init__(
        self, 
        filepaths: np.ndarray, 
        size: int,
        labels: Union[np.ndarray, None]=None, 
        transform=None
    ):
        
        self.filepaths = filepaths
        self.labels = labels
        self.size = size
        self.transform = transform
    
    def __len__(self):
        return self.filepaths.shape[0]
    
    def __getitem__(self, idx):
        image = get_image(self.filepaths[idx], self.size)
        if self.labels is None:
            return self.transform(image)
        return self.transform(image), torch.FloatTensor(self.labels[idx])

### **Model**

In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
            
        self.model = timm.create_model(model_name="efficientnet_b4", pretrained=False)
        self.model.classifier = nn.Linear(in_features=self.model.classifier.in_features, out_features=1)

    def forward(self, x):
        return self.model(x)

### **Fit and Predict**

In [7]:
# def fit(
#     model=None,
#     optimizer=None, 
#     scheduler_rlrop=None,
#     scheduler_oclr=None,
#     epochs=None, 
#     early_stopping_patience=None, 
#     dataloaders=None, 
#     fold=None, 
#     save_path=None,
#     device=None,
#     verbose=False
# ) -> tuple:
    
#     def get_accuracy(y_pred, y_true):
#         y_pred = torch.sigmoid(y_pred)
        
#         y_pred[y_pred > 0.5] = 1
#         y_pred[y_pred <= 0.5] = 0
        
#         return torch.count_nonzero(y_pred == y_true).item() / len(y_pred)
    
    
#     if verbose:
#         breaker()
#         if fold: print(f"Training Fold {fold}...")
#         else: print("Training ...")
#         breaker()
        
#     bestLoss: dict = {"train" : np.inf, "valid" : np.inf} 
#     bestAccs: dict = {"train" : 0.0, "valid" : 0.0}
    
#     Losses: list = []
#     Accuracies: list = [] 
#     LRs: list = []
        
#     if fold: 
#         ble_name = f"ble_state_fold_{fold}.pt"
#         bae_name = f"bae_state_fold_{fold}.pt"
#     else: 
#         ble_name = f"ble_state.pt"
#         bae_name = f"bae_state.pt"
        
#     start_time = time()
#     for e in range(epochs):
#         e_st = time()
#         epochLoss: dict = {"train" : 0.0, "valid" : 0.0} 
#         epochAccs: dict = {"train" : 0.0, "valid" : 0.0}

#         for phase in ["train", "valid"]:
#             if phase == "train":
#                 model.train()
#             else:
#                 model.eval()
            
#             lossPerPass: list = []
#             accsPerPass: list = []
                
#             for X, y in dataloaders[phase]:
#                 X, y = X.to(device), y.to(device)

#                 optimizer.zero_grad()
#                 with torch.set_grad_enabled(phase == "train"):
#                     output = model(X)
#                     loss = nn.BCEWithLogitsLoss()(output, y)
#                     if phase == "train":
#                         loss.backward()
#                         optimizer.step()
#                         if scheduler_oclr: scheduler_oclr.step()
#                 lossPerPass.append(loss.item())
#                 accsPerPass.append(get_accuracy(output, y))
#             epochLoss[phase] = np.mean(np.array(lossPerPass))
#             epochAccs[phase] = np.mean(np.array(accsPerPass))
#         if scheduler_oclr: LRs.append(scheduler_oclr.get_last_lr())
#         Losses.append(epochLoss)
#         Accuracies.append(epochAccs)
        
#         if scheduler_oclr:
#             save_dict = {"model_state_dict"     : model.state_dict(),
#                          "optim_state_dict"     : optimizer.state_dict(),
#                          "scheduler_state_dict" : scheduler_oclr.state_dict()}
        
#         elif scheduler_rlrop:
#             save_dict = {"model_state_dict"     : model.state_dict(),
#                          "optim_state_dict"     : optimizer.state_dict(),
#                          "scheduler_state_dict" : scheduler_rlrop.state_dict()}
        
#         else:
#             save_dict = {"model_state_dict"     : model.state_dict(),
#                          "optim_state_dict"     : optimizer.state_dict()}
        
#         if early_stopping_patience:
#             if epochLoss["valid"] < bestLoss["valid"]:
#                 bestLoss = epochLoss
#                 BLE = e + 1
#                 torch.save(save_dict, os.path.join(save_path, ble_name))
#                 torch.save(save_dict, os.path.join(save_path, bae_name))
#                 early_stopping_step = 0
#             else:
#                 early_stopping_step += 1
#                 if early_stopping_step > early_stopping_patience:
#                     print("\nEarly Stopping at Epoch {}".format(e + 1))
#                     break
        
#         if epochLoss["valid"] < bestLoss["valid"]:
#             bestLoss = epochLoss
#             BLE = e + 1
#             torch.save(save_dict,os.path.join(save_path, ble_name))
        
#         if epochAccs["valid"] > bestAccs["valid"]:
#             bestAccs = epochAccs
#             BAE = e + 1
#             torch.save(save_dict,os.path.join(save_path, bae_name))
        
#         if scheduler_rlrop: scheduler_rlrop.step(epochLoss["valid"])
        
#         if verbose:
#             print("Epoch: {} | Train Loss: {:.5f} | Valid Loss: {:.5f} |\
#  Train Accs: {:.5f} | Valid Accs: {:.5f} | Time: {:.2f} seconds".format(e+1, 
#                                                                         epochLoss["train"], epochLoss["valid"], 
#                                                                         epochAccs["train"], epochAccs["valid"], 
#                                                                         time()-e_st))

#     if verbose:                                           
#         breaker()
#         print(f"Best Validation Loss at Epoch {BLE}")
#         breaker()
#         print(f"Best Validation Accs at Epoch {BAE}")
#         breaker()
#         print("Time Taken [{} Epochs] : {:.2f} minutes".format(len(Losses), (time()-start_time)/60))
    
#     return Losses, Accuracies, LRs, bestLoss, bestAccs, BLE, BAE, ble_name, bae_name


def predict_batch(model=None, dataloader=None, path=None, device=None) -> np.ndarray:
    model.load_state_dict(torch.load(path, map_location=device)["model_state_dict"])
    model.to(device)    
    model.eval()
    
    y_pred = torch.zeros(1, 1).to(device)
    
    for X in dataloader:
        X = X.to(device)
        with torch.no_grad():
            output = torch.sigmoid(model(X))
        y_pred = torch.cat((y_pred, output.view(-1, 1)), dim=0)
    
    return y_pred[1:].detach().cpu().numpy()

### **Train**

In [8]:
# df = pd.read_csv("/kaggle/input/scv-dataframe/train.csv")

# filepaths = df.filepaths.copy().values
# labels = df.labels.copy().values

In [9]:
# fold: int = 1
# BLs: list = []
# BAs: list = []
    
# cfg.batch_size = 64
# cfg.epochs = 10
    
# for tr_idx, va_idx in KFold(n_splits=cfg.n_splits, random_state=cfg.seed, shuffle=True).split(filepaths):

#     tr_filepaths, va_filepaths = filepaths[tr_idx], filepaths[va_idx] 
#     tr_labels, va_labels       = labels[tr_idx], labels[va_idx]

#     tr_data_setup = DS(
#         filepaths=tr_filepaths, 
#         labels=tr_labels.reshape(-1, 1),
#         size=cfg.size,
#         transform=cfg.train_transform
#     )

#     va_data_setup = DS(
#         filepaths=va_filepaths, 
#         labels=va_labels.reshape(-1, 1),
#         size=cfg.size,
#         transform=cfg.train_transform
#     )

#     dataloaders = {
#         "train" : DL(tr_data_setup, batch_size=cfg.batch_size, shuffle=True, generator=torch.manual_seed(cfg.seed)),
#         "valid" : DL(va_data_setup, batch_size=cfg.batch_size, shuffle=False),
#     }

#     cfg.steps_per_epoch=len(dataloaders["train"])

#     torch.manual_seed(cfg.seed)
#     model = Model().to(cfg.device)

#     optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=cfg.lr, weight_decay=cfg.wd)
#     # optimizer = optim.AdamW([p for p in model.parameters() if p.requires_grad], lr=cfg.lr, weight_decay=cfg.wd)
#     # optimizer = optim.SGD([p for p in model.parameters() if p.requires_grad], lr=cfg.lr, weight_decay=cfg.wd, momentum=0.9)

#     scheduler_oclr = optim.lr_scheduler.OneCycleLR(
#         optimizer=optimizer, 
#         max_lr=cfg.max_lr, 
#         epochs=cfg.epochs, 
#         steps_per_epoch=cfg.steps_per_epoch,
#         pct_start=cfg.pct_start, 
#         div_factor=cfg.div_factor, 
#         final_div_factor=cfg.final_div_factor
#     )

#     # scheduler_rlrop = optim.lr_scheduler.ReduceLROnPlateau(
#     #     optimizer=optimizer,
#     #     patience=cfg.patience,
#     #     eps=cfg.eps,
#     #     verbose=True
#     # )

#     # scheduler_oclr = None
#     scheduler_rlrop = None


#     L, A, LR, BL, BA, _, _, _, _ = fit(
#         model=model, 
#         optimizer=optimizer, 
#         scheduler_oclr=scheduler_oclr,
#         scheduler_rlrop=scheduler_rlrop,
#         epochs=cfg.epochs, 
#         early_stopping_patience=cfg.early_stopping, 
#         dataloaders=dataloaders, 
#         device=cfg.device,
#         save_path=cfg.save_path,
#         fold=fold,
#         verbose=True
#     )


#     breaker()
#     show_loss_graphs(L)
#     breaker()
#     show_accuracy_graphs(A)
#     breaker()
#     if scheduler_oclr:
#         show_lr_graph(LR)
#         breaker()

#     BLs.append(BL)
#     BAs.append(BA)

#     fold += 1

### **Best Model**

In [10]:
# BL = np.inf
# for i in range(len(BLs)):
#     if BLs[i]["valid"] < BL:
#         BL = BLs[i]["valid"]
#         best_loss_index = i

        
# BA = 0.0
# for i in range(len(BAs)):
#     if BAs[i]["valid"] > BA:
#         BA = BAs[i]["valid"]
#         best_accs_index = i

# breaker()
# print(f"Best Loss Model Fold     : {best_loss_index + 1}")
# print(f"Best Accuracy Model Fold : {best_accs_index + 1}")
# breaker()

### **Submission**

In [11]:
ts_filepaths = pd.read_csv("/kaggle/input/scv-dataframe/test.csv").filepaths.copy().values
ts_data_setup = DS(
    filepaths=ts_filepaths, 
    labels=None,
    size=cfg.size,
    transform=cfg.valid_transform
)
ts_data = DL(ts_data_setup, batch_size=64, shuffle=False)

model = Model().to(cfg.device)

In [12]:
y_pred_1_l = predict_batch(
    model=model,
    dataloader=ts_data,
    path="/kaggle/input/scv-en4-a384-e10/saves/ble_state_fold_1.pt",
    device=cfg.device
)

y_pred_2_l = predict_batch(
    model=model,
    dataloader=ts_data,
    path="/kaggle/input/scv-en4-a384-e10/saves/ble_state_fold_2.pt",
    device=cfg.device
)

y_pred_3_l = predict_batch(
    model=model,
    dataloader=ts_data,
    path="/kaggle/input/scv-en4-a384-e10/saves/ble_state_fold_3.pt",
    device=cfg.device
)

y_pred_4_l = predict_batch(
    model=model,
    dataloader=ts_data,
    path="/kaggle/input/scv-en4-a384-e10/saves/ble_state_fold_4.pt",
    device=cfg.device
)

y_pred_5_l = predict_batch(
    model=model,
    dataloader=ts_data,
    path="/kaggle/input/scv-en4-a384-e10/saves/ble_state_fold_5.pt",
    device=cfg.device
)

In [13]:
y_pred_1_a = predict_batch(
    model=model,
    dataloader=ts_data,
    path="/kaggle/input/scv-en4-a384-e10/saves/bae_state_fold_1.pt",
    device=cfg.device
)

y_pred_2_a = predict_batch(
    model=model,
    dataloader=ts_data,
    path="/kaggle/input/scv-en4-a384-e10/saves/bae_state_fold_2.pt",
    device=cfg.device
)

y_pred_3_a = predict_batch(
    model=model,
    dataloader=ts_data,
    path="/kaggle/input/scv-en4-a384-e10/saves/bae_state_fold_3.pt",
    device=cfg.device
)

y_pred_4_a = predict_batch(
    model=model,
    dataloader=ts_data,
    path="/kaggle/input/scv-en4-a384-e10/saves/bae_state_fold_4.pt",
    device=cfg.device
)

y_pred_5_a = predict_batch(
    model=model,
    dataloader=ts_data,
    path="/kaggle/input/scv-en4-a384-e10/saves/bae_state_fold_5.pt",
    device=cfg.device
)

In [14]:
y_pred = (
    y_pred_1_l + y_pred_2_l + y_pred_3_l + y_pred_4_l + y_pred_5_l + y_pred_1_a + y_pred_2_a + y_pred_3_a + y_pred_4_a + y_pred_5_a
) / 10

ss_df = pd.read_csv("/kaggle/input/shift-cv-2023/sample_submission.csv")
ss_df.blur = y_pred
ss_df.to_csv("submission.csv", index=False)